In [1]:
import shlex, subprocess
import re
import os 

In [3]:
def number_images(route_images):
    """Extract the number of images from a directory.
    
    Args:
    route_images (str): Route of images.
    
    Returns:
    N (int): Number of images.
    
    """
    path, dirs, files = os.walk(route_images).__next__()
    N = len(files)
    return N

N = number_images('tests/img') 


In [ ]:

def Audio_festival(route_text, route_audio, text, language, N):
    """It generates audios from ffmpeg and texts.
    
    Args:
    route_text (str): Route of text files. 
    route_audio (str): Route of audio. 
    text (list): Name of each text.
    language (str): Voice for ffmpeg.
    N (int): Number of images necessarily equal to the number of text files.
    
    Returns:
    N audio files
    
    """
    log = open('process.log', 'a') # File log 
    log.write('Audio_festival')
    log.flush()
    
    print('Text to Audio....')
    print(language)

    for i in range(N):
        
        command_line ='text2wave '+route_text + '/' + text[i] + ' -o ' \
            + route_audio + '/0%d.wav '%i+' '+' -eval ''('+language+')'''   
        args = shlex.split(command_line)
        c = subprocess.Popen(args, stdout=log, stderr=log)
        
# For language see available festival voices

Audio_festival('tests/text','tests/audio',['01.text','02.txt','03.txt'],'don_diphone', N)

In [ ]:
def duration(N, route_images, route_audio, img):
    """Get the duration of each audio file.
    
    Args:
    N (int): Number of images .
    route_images (str): Route of images.
    route_audio (str): Route of audio.
    img (list): Name of the images.
    
    Returns:
    Duration.txt (file): Generates a file with the duration of
    each audio,with the format specified by ffmpeg.
    
    """
    f= open('Duration.txt','w')
    f.write('ffconcat version 1.0\n')
    
    for i in range(N):
        
        f.write('file '+route_images+'/'+img[i]+' \n')
        process = subprocess.Popen(['ffmpeg', '-i', route_audio+'/0%d.wav'%i], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        out, _ = process.communicate()
        result = re.search(r"Duration:\s{1}(?P<hours>\d+?):(?P<minutes>\d+?):(?P<seconds>\d+\.\d+?),", out.decode('utf-8'), re.DOTALL).groupdict()
        f.write('duration  '+result['seconds']+'\n')
        
    f.close()

duration(N,'tests/img','tests/audio',['01.jpg','02.jpg','03.jpg'])


In [ ]:
def image_video(r_video):
    """Generating video from images and archive duration
    with format specifited by ffmpeg.
    
    Args:
    r_video (str): Route of video.
    
    Returns:
    Video0.mp4 (video): Video of images each with specific duration.
    
    """
    log = open('process.log', 'a') # File log 
    log.write('image_video\n')
    log.flush()  
    print('Image to video...')
    
    
    command_line = 'ffmpeg -i Duration.txt -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2"  '+r_video+'/Video0.mp4'
    args = shlex.split(command_line)
    c = subprocess.Popen(args, stdout=log, stderr=log)
    
image_video('tests')

In [ ]:
def u_audios(N,route_audio):
    """Concatenate audios.
    
    Args:
    N (int): Number of images.
    route_audio (str): Route of audio files.
    
    Returns:
    audio.wav (file): Audio union.
    Aud.txt (file): Archivo for  union of audios with ffmpeg.
    
    """
    log = open('process.log', 'a')
    log.write('U_Audios\n')
    log.flush()
    print('Concatenating audios')
    
    f = open('Aud.txt', 'w')
    
    for i in range(N):
        
        f.write('file '+route_audio+'/0%d.wav \n'%i)
    f.close()
    command_line = 'ffmpeg -f concat -safe 0 -i Aud.txt -c copy ' + route_audio \
        + '/audio.wav'
    args = shlex.split(command_line)
    c = subprocess.Popen(args, stdout=log, stderr=log)
    
u_audios(N,'tests/audio')

In [ ]:
def video_u_audio(route_audio, route_video, name_video):
    """Join audio and video.
    
    Args:
    route_audio (str): Route audio.
    route_video (str): Rote video.
    name_video (str): Output video name.
    
    Returns:
    video (file): Video. 
    
    """
    log = open('process.log', 'a')
    log.write('Video_U_Audio\n')
    log.flush() 
    print('Video.............')
    
    command_line = 'ffmpeg -i '+route_video+'/Video0.mp4 -i '+route_audio+'/audio.wav -strict -2 '+route_video+'/'+name_video
    args = shlex.split(command_line)
    c = subprocess.Popen(args,stdout=log, stderr=log).communicate()


video_u_audio('tests/audio','tests','guardianes.mp4')    